<a href="https://colab.research.google.com/github/stfc/janus-tutorials/blob/main/phonons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phonon calculations with machine learnt interatomic potentials

### Setup environment (optional)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!python3 -m pip install -U git+https://github.com/stfc/janus-core.git
!python3 -m pip install -U git+https://gitlab.com/ase/ase.git
!python3 -m pip install matgl
!python3 -m pip install chgnet

## Phonons (periodic)

In [ ]:
from ase.build import bulk

from janus_core.calculations.single_point import SinglePoint
from janus_core.calculations.phonons import Phonons

### Prepare for phonon calculations on salt 

In [ ]:
NaCl = bulk('NaCl', 'rocksalt', a=5.63, cubic=True)

In [ ]:
sp_mace = SinglePoint(
    struct=NaCl.copy(),
    architecture="mace_mp",
    device='cpu',
    calc_kwargs={'model_paths':'small','default_dtype':'float64'}
)

Note: Set `filter_func = None` for geometry optimization via `minimize_kwargs`, so cell is fixed

In [ ]:
phonons_mace = Phonons(
    struct=sp_mace.struct,
    supercell=[2, 2, 2],
    displacement=0.01,
    t_step=10.0,
    t_min=0.0,
    t_max=1000.0,
    minimize=True,
    hdf5=True,
    plot_to_file=True,
    symmetrize=False,
    write_full=True,
    minimize_kwargs={"filter_func": None},
)

Optimize structure and calculate force constants using phonopy.

This will save phonopy to `Cl4Na4-phonopy.yml`, and additionally save force constants to `Cl4Na4-force_constants.hdf5`:

In [ ]:
phonons_mace.calc_force_constants()

Check cell parameters have not been changed by optimization:

In [ ]:
print(phonons_mace.struct.cell.cellpar())

Calculate and plot band structure, writing results to `Cl4Na4-auto_bands.yml`, and saving the figure as `Cl4Na4-auto_bands.svg`:

In [ ]:
phonons_mace.calc_bands(write_results=True)

Calculate thermal properties, saving the heat capacity, enthalpy, and entropy, to `Cl4Na4-thermal.dat`:

In [ ]:
phonons_mace.calc_thermal_props()

### Phonon calcualtions with optimization of cell

The same calculations can be run with cell lengths, but not angles, optimized:

In [ ]:
sp_mace_lengths_only = SinglePoint(
    struct=NaCl.copy(),
    architecture="mace_mp",
    device='cpu',
    calc_kwargs={'model_paths': 'small', 'default_dtype': 'float64'}
)

Note: Set `"filter_kwargs" = {"hydrostatic_strain": True}` for geometry optimization via `minimize_kwargs`, so cell angles are fixed, but lengths can change

In [ ]:
phonons_mace_lengths_only = Phonons(
    struct=sp_mace_lengths_only.struct,
    supercell=[2, 2, 2],
    displacement=0.01,
    t_step=10.0,
    t_min=0.0,
    t_max=1000.0,
    minimize=True,
    hdf5=True,
    plot_to_file=True,
    symmetrize=False,
    write_full=True,
    minimize_kwargs={"filter_kwargs": {"hydrostatic_strain": True}},
)

In [ ]:
phonons_mace_lengths_only.calc_bands(write_results=True)

Confirm changes to cell lengths:

In [ ]:
print(phonons_mace_lengths_only.struct.cell.cellpar())

### Phonon calculations with pressure

Calculations can also be run at a fixed pressure, as well as optmising both the cell lengths and angles

In [ ]:
sp_mace_pressure = SinglePoint(
    struct=NaCl.copy(),
    architecture="mace_mp",
    device='cpu',
    calc_kwargs={'model_paths':'small','default_dtype':'float64'}
)

Note: Set `"filter_kwargs" = {"scalar_pressure": x}` for geometry optimization via `minimize_kwargs` to set the pressure. Without setting `hydrostatic_strain =  True`, both the cell lengths and angles will be optimized 

In [ ]:
phonons_mace_pressure = Phonons(
    struct=sp_mace_pressure.struct,
    supercell=[2, 2, 2],
    displacement=0.01,
    t_step=10.0,
    t_min=0.0,
    t_max=1000.0,
    minimize=True,
    hdf5=True,
    plot_to_file=True,
    symmetrize=False,
    write_full=True,
    minimize_kwargs={"filter_kwargs": {"scalar_pressure": 0.1}},
)

In [ ]:
phonons_mace_pressure.calc_bands(write_results=True)

Confirm changes to cell:

In [ ]:
print(phonons_mace_pressure.struct.cell.cellpar())

Compare band structures for different optimization options and save to files:

In [ ]:
phonons_mace.write_band_structure(plot_file="NaCl_mace.svg")
phonons_mace_lengths_only.write_band_structure(plot_file="NaCl_lengths_only.svg")
phonons_mace_pressure.write_band_structure(plot_file="NaCl_pressure.svg")

## Comparing the band structure from MACE to CHGNet and M3GNet:

### Calculate band structure using CHGNET

In [ ]:
sp_chgnet = SinglePoint(
    struct=NaCl.copy(),
    architecture="chgnet",
    device="cpu"
)

In [ ]:
phonons_chgnet = Phonons(
    struct=sp_chgnet.struct,
    supercell=[2, 2, 2],
    displacement=0.01,
    t_step=10.0,
    t_min=0.0,
    t_max=1000.0,
    minimize=True,
    hdf5=True,
    plot_to_file=True,
    symmetrize=False,
    write_full=True,
    minimize_kwargs={"filter_func": None},
)

In [ ]:
phonons_chgnet.calc_bands(write_results=True)

### Calculate band structure using M3GNET

In [ ]:
sp_m3gnet = SinglePoint(
    struct=NaCl.copy(),
    architecture="m3gnet",
    device="cpu"
)

In [ ]:
phonons_m3gnet = Phonons(
    struct=sp_m3gnet.struct,
    supercell=[2, 2, 2],
    displacement=0.01,
    t_step=10.0,
    t_min=0.0,
    t_max=1000.0,
    minimize=True,
    hdf5=True,
    plot_to_file=True,
    symmetrize=False,
    write_full=True,
    minimize_kwargs={"filter_func": None},
)

In [ ]:
phonons_m3gnet.calc_bands(write_results=True)

Compare and save plots for each MLIP:

In [ ]:
phonons_mace.write_band_structure(plot_file="MACE.svg")
phonons_chgnet.write_band_structure(plot_file="chgnet.svg")
phonons_m3gnet.write_band_structure(plot_file="m3gnet.svg")

Note: It may be necessary to reset the default PyTorch dtype if different calculators have been set:

In [ ]:
import torch
torch.set_default_dtype(torch.float64)

phonons_mace.calc_force_constants()